In [ ]:
import mlrun
from sys import path



project_name_base = 'widsdb2'
project_path = 'conf'
widsdb2_proj = mlrun.projects.load_project(project_path,  clone=True)

project = widsdb2_proj



In [ ]:
names = [func.get('metadata').get('name')
         for func in mlrun.get_run_db().list_functions(project={project.name}, tag='latest')
         if func.get('kind') != '']
print(names)

In [4]:
project.set_function(f'db://{project.name}/serving')

In [3]:
project = widsdb2_proj
names = [f.get('name') for f in widsdb2_proj.functions]
print(names)
#if func.get('kind') != ''
#tag='latest'

['prep', 'tstprep', 'train-wids', 'describe', 'test-classifier']


In [4]:
[f.get('name') for f in widsdb2_proj.functions]

['prep', 'train-wids', 'describe', 'test-classifier', 'tstprep']

In [7]:
fn = f'store://widsdb2/train-wids-train_wids_model'
  

In [8]:
f1 = f'store://widsdb2/serving'


In [5]:

import mlrun
import pandas as pd
from mlrun.run import get_dataitem


train_df = pd.read_csv('TrainingWiDS2021.csv', index_col=[0])
y = train_df.diabetes_mellitus.values
#del train_df['diabetes_mellitus']
print(train_df.shape)
widsdb2_proj.log_dataset(key='raw_train_data', df=train_df, index=False, format='csv')




(130157, 180)


In [5]:
%%writefile {'conf/workflow.py' }

from kfp import dsl
import mlrun
from mlrun.platforms import auto_mount


funcs = {}
DATASET = 'train_enc'
LABELS =  'diabetes_mellitus'

# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    for f in functions.values():
        f.apply(auto_mount())

# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(
    name="WidsDB2",
    description="This workflow implements the pipeline for data preprocessing, training model "
                "serving for Widsdb2 dataset \n"
                
)

def kfpipeline(source_url='store://raw_train_data'):

    # Ingest the data set
    ingest = funcs['prep'].as_step(
        name="prep",
        handler='trdata_prep',
        inputs={'src': source_url},
        outputs=[DATASET])
    
    # Train a model   
    train = funcs["train-wids"].as_step(
        name="train-wids",
        params={"label_column": LABELS},
        inputs={"dataset": ingest.outputs[DATASET]},
        outputs=['model', 'test_set'])
    
    # Test and visualize the model
    test = funcs["test-classifier"].as_step(
        name="test-classifier",
        params={"label_column": LABELS},
        inputs={"models_path": train.outputs['model'],
                "test_set": train.outputs['test_set']})
    
    # Deploy the model as a serverless function
    deploy = funcs["serving"].deploy_step(
        models={f"{DATASET}_v1": train.outputs['model']})

Overwriting conf/workflow.py


In [7]:
# Register the workflow file as "main"
project.set_workflow('main', 'workflow.py')

In [8]:
project.save()

In [9]:

import os 
from os import environ, path
from mlrun import mlconf



In [10]:
 mlconf.artifact_path



'v3io:///projects/{{run.project}}/artifacts'

In [11]:
pipeline_path = mlconf.artifact_path

run_id = project.run(
    'main',
    arguments={}, 
    artifact_path=os.path.join(pipeline_path, "pipeline", '{{workflow.uid}}'),
    dirty=True,
    watch=True)

> 2021-06-21 16:41:05,710 [info] using in-cluster config.


> 2021-06-21 16:41:06,846 [info] Pipeline run id=f545d0df-9e75-42ec-bec7-63dc029f50bb, check UI or DB for progress
> 2021-06-21 16:41:06,847 [info] waiting for pipeline run completion


RuntimeError: run status Failed not in expected statuses

In [27]:
dfa = mlrun.get_dataitem('store://raw_train_data')

OSError: artifact store://raw_train_data not found, read artifact default/raw_train_data